# Chatper 8: Graph Search and its Applications

## Breadth First Search (BFS)

<div>
<img src="img/fig8.5.jpg" width="600"/>
</div>

In [23]:
graph1 = {
    's': ('a', 'b'),
    'a': ('c', 's'),
    'b': ('c', 'd', 's'),
    'c': ('a', 'b', 'd', 'e'),
    'd': ('b', 'c', 'e'),
    'e': ('c', 'd')
}

In [24]:
from collections import deque

def bfs(graph, start):
    # explored = {start}
    explored = [start]
    queue = deque([start])
    while queue:
        v = queue.pop()
        for w in graph[v]:
            if w not in explored:
                # explored.add(w)
                explored.append(w)
                queue.appendleft(w)
    return explored

In [25]:
bfs(graph1, 's')

['s', 'a', 'b', 'c', 'd', 'e']

### Two disjoint graphs

In [26]:
graph2 = {
    's': ('a', 'b'),
    'a': ('c', 's'),
    'b': ('c', 'd', 's'),
    'c': ('a', 'b', 'd', 'e'),
    'd': ('b', 'c', 'e'),
    'e': ('c', 'd'),
    # Disconnected part
    'f': ('g', 'h'),
    'g': ('f', 'h'),
    'h': ('f', 'g')
}

In [27]:
bfs(graph2, 's')

['s', 'a', 'b', 'c', 'd', 'e']

In [28]:
bfs(graph2, 'g')

['g', 'f', 'h']

### Shortest Path

If $v$ is in layer $k$ and $w$ is in layer $k+1$, then $\mathrm{dist}(s, w) = \mathrm{dist}(s, v) + 1$. This is the same as saying that $w$ is an unexplored neighbor of $v$, its distance is $\mathrm{dist}(s, v) + 1$.

In [29]:
from math import inf

def shortest_path(graph, start):
    # Note the initialization of the dictionary containing
    # the distances from the start node.
    dist = {k: inf if k != start else 0 for k in graph}
    # explored = {start}
    explored = [start]
    queue = deque([start])
    while queue:
        v = queue.pop()
        for w in graph[v]:
            if w not in explored:
                # explored.add(w)
                explored.append(w)
                dist[w] = dist[v] + 1
                queue.appendleft(w)
    return explored, dist

In [30]:
res, dists = shortest_path(graph1, 's')
dists

{'s': 0, 'a': 1, 'b': 1, 'c': 2, 'd': 2, 'e': 3}

### Connected Components of an Undirected Graph

The idea behind finding the connected components in an undirected graph (in a directed graph it is more difficult), is the following:

1. Start with a counter set to 0. No node is explored.
2. Iterate over every node in the graph:

In [31]:
def conn_comp(graph):
    comp = {k: -1 for k in graph}
    # The number of components is initialized to 0, not 1.
    ncc = 0
    # explored = set()
    explored = []
    for s in graph:
        if s not in explored:
            ncc += 1
            queue = deque([s])
            while queue:
                v = queue.pop()
                comp[v] = ncc
                for w in graph[v]:
                    if w not in explored:
                        # explored.add(w)
                        explored.append(w)
                        queue.appendleft(w)
    return ncc, comp

In [32]:
conn_comp(graph1)

(1, {'s': 1, 'a': 1, 'b': 1, 'c': 1, 'd': 1, 'e': 1})

In [33]:
conn_comp(graph2)

(2, {'s': 1, 'a': 1, 'b': 1, 'c': 1, 'd': 1, 'e': 1, 'f': 2, 'g': 2, 'h': 2})

## Depth First Search (DFS)

<div>
<img src="img/dfs.jpg" width=600/>
</div>

The two main differences from BFS are:

1. DFS uses a stack, while BFS uses a queue.
2. DFS checks if a node was already explored only after removing it from the stack.

This second point is non-trivial. A thorough discussion can be found in [David Eppstein's blog](https://11011110.github.io/blog/2013/12/17/stack-based-graph-traversal.html). In what follows, we will compare the results of BFS, DFS, and what he calls Stack-Based Graph Traversal (SBGT). See also the [Wikipedia page on Depth-First Search](https://en.wikipedia.org/wiki/Depth-first_search).

**Conjecture to be verified** the difference between DFS and SBGT is how the frontier is defined.

In [35]:
def dfs(graph, start):
    # explored is initially empty.
    # explored = set()
    explored = []
    # We use a stack, initialized to start
    stack = [start]
    while stack:
        v = stack.pop()
        if v not in explored:
            # explored.add(v)
            explored.append(v)
            for w in graph[v]:
                stack.append(w)
    return explored

In [36]:
dfs(graph1, 's')

['s', 'b', 'd', 'e', 'c', 'a']

Let's compare BFS and DFS on a tree

In [37]:
tree = {'s': ['a', 'b'],
        'a': ['s', 'c', 'd'],
        'c': ['a'],
        'd': ['a'],
        'b': ['s']}
print(f"BFS: {bfs(tree, 's')}")
print(f"DFS: {dfs(tree, 's')}")

BFS: ['s', 'a', 'b', 'c', 'd']
DFS: ['s', 'b', 'a', 'd', 'c']


We also implement the incorrect version of DFS obtained by just replacing the queue with a stack in BFS, i.e, the SGBT, as shown [here](https://11011110.github.io/blog/2013/12/17/stack-based-graph-traversal.html).

In [38]:
def sbgt(graph, start):
    """Stack based graph traversal."""
    # explored = {start}
    explored = [start]
    stack = [start]
    while stack:
        v = stack.pop()
        for w in graph[v]:
            if w not in explored:
                # explored.add(w)
                explored.append(w)
                stack.append(w)
    return explored

For comparison, we show the sequence of explorations from BFS, DFS, and SBGT on `graph1`...

In [39]:
print(f"BFS:  {bfs(graph1, 's')}")
print(f"DFS:  {dfs(graph1, 's')}")
print(f"SBGT: {sbgt(graph1, 's')}")

BFS:  ['s', 'a', 'b', 'c', 'd', 'e']
DFS:  ['s', 'b', 'd', 'e', 'c', 'a']
SBGT: ['s', 'a', 'b', 'c', 'd', 'e']


... and on `tree`

In [40]:
print(f"BFS:  {bfs(tree, 's')}")
print(f"DFS:  {dfs(tree, 's')}")
print(f"SBGT: {sbgt(tree, 's')}")

BFS:  ['s', 'a', 'b', 'c', 'd']
DFS:  ['s', 'b', 'a', 'd', 'c']
SBGT: ['s', 'a', 'b', 'c', 'd']


Eppstein shows the difference using the graph below.

<div>
<img src="img/eppstein.jpg" width="300"/>
</div>

In [41]:
eppstein = {
    's': ['a', 'c'],
    'a': ['s', 'b', 'd'],
    'b': ['a', 'e'],
    'c': ['s', 'd', 'f'],
    'd': ['a', 'c', 'e', 'g'],
    'e': ['b', 'd', 'h'],
    'f': ['c', 'g'],
    'g': ['d', 'f', 'h'],
    'h': ['e', 'g']
}

print(f"BFS:  {bfs(eppstein, 's')}")
print(f"DFS:  {dfs(eppstein, 's')}")
print(f"SBGT: {sbgt(eppstein, 's')}")

BFS:  ['s', 'a', 'c', 'b', 'd', 'f', 'e', 'g', 'h']
DFS:  ['s', 'c', 'f', 'g', 'h', 'e', 'd', 'a', 'b']
SBGT: ['s', 'a', 'c', 'd', 'f', 'g', 'h', 'e', 'b']


In this example the three traversals are all different. The set of explored nodes is the same, but the order in which they are visited is different.

### DFS: Recursive Implementation With an External Set

The recurrent version requires an external set. In this it looks like we are not using a stack, but this is not the case: we use the stack frame created by the recursive calls.

In [16]:
explored = set()

def dfs_rec1(graph, start):
    explored.add(start)
    for v in graph[start]:
        if v not in explored:
            dfs_rec1(graph, v)

In [17]:
print("Before: ", explored)
dfs_rec1(graph1, 's')
print("After:  ", explored)

Before:  set()
After:   {'a', 'e', 's', 'b', 'c', 'd'}


### DFS: Recursive Implementation Without an External Set

Rather than having an empty set hanging around, it is safer to wrap the function into another function.

In [18]:
def dfs_rec2(graph, start):
    explored = set()
    def dfs_inner(graph, start):
        explored.add(start)
        for v in graph[start]:
            if v not in explored:
                dfs_inner(graph, v)
    dfs_inner(graph, start)
    return explored

In [19]:
explored = set()  # Just to be sure
res = dfs_rec2(graph1, 's')
res

{'a', 'b', 'c', 'd', 'e', 's'}

### Topological Sort

Given a graph $G(V, E)$, a *topological ordering* is a function on the vertices $v \in V$ such that for every edge $(v, w) \in E$, $f(v) < f(w)$.

#### Topological Sort via DFS

Note that we have to use the `nonlocal` keyword. A better approach is probably using a class and implementing this as a method. You can read more on non-local and global variables on Real Python [here](https://realpython.com/python-scope-legb-rule/#the-nonlocal-statement) and [here](https://realpython.com/python-use-global-variable-in-function/).

In [20]:
def toposort(graph):
    explored = set()
    ordering = {}
    curlabel = len(graph.keys())

    def dfs_topo(graph, s):
        nonlocal curlabel
        explored.add(s)
        for w in graph[s]:
            if w not in explored:
                dfs_topo(graph, w)
        ordering[s] = curlabel
        curlabel -= 1

    for v in graph:
        if v not in explored:
            dfs_topo(graph, v)
    return ordering

In [21]:
toposort(graph1)

{'e': 6, 'd': 5, 'b': 4, 'c': 3, 'a': 2, 's': 1}